# MagNet: Model the Geomagnetic Field Chapter 1
## Develop the CNN Model


![HELIO_GRAPHIC_URL](https://ngdc.noaa.gov/geomag/img/challenge-banner.png "HELIO")

* Creator(s): Rob.Redmon@noaa.gov (1,2), Manoj.C.Nair@noaa.gov (2,3), LiYin.Young@noaa.gov (2,3)
* Affiliation(s):
    * 1. National Centers for Environmental Information ([NCEI](https://www.ncei.noaa.gov/)), National Oceanic and Atmospheric Administration (NOAA),
    * 2. NOAA Center for Artificial Intelligence ([NCAI](https://noaa.gov/ai)),
    * 3. Cooperative Institute for Research for Environmental Sciences [CIRES](https://cires.colorado.edu/).
* History
    * 2023-08: Content reorganized for the [NCAI](https://noaa.gov/ai) <i>Learning Journey</i> library. No significant technical changes from previous version.
    * 2022-06: Initial notebook version developed for the [TAI4ES 2022 Summer School](https://www2.cisl.ucar.edu/events/tai4es-2022-summer-school).
* Acknowledgements:
    * Original funding support was provided by the NCEI Innovates program.
    * Post-model inference and evaluation were created for the NCAR and [AI2ES](https://www.ai2es.org/) [TAI4ES 2022 Summer School](https://www2.cisl.ucar.edu/events/tai4es-2022-summer-school).
    * Feature exploration through model training was inspired by the benchmark [DrivenData blogpost](https://www.drivendata.co/blog/model-geomagnetic-field-benchmark/), developed for NOAA's [MagNet competition](https://ngdc.noaa.gov/geomag/mag-net-challenge.html).

## Overview
<b> Chapter 1 "Develop the CNN Model"</b> of the two notebook series, provides the benchmark machine learning modeling experience for a key space weather storm indicator, the disturbance-storm-time (<i>Dst</i>) index, for the 2020 NOAA competition, "MagNet: Model the Geomagnetic Field".  This notebook is based on the second-place solution to the MagNet competition held by the National Oceanic and Atmospheric Administration (NOAA) and National Aeronautics and Space Administration (NASA) in 2020-21.


### Prerequisites

* <b>Python intermediate proficiency for data science:</b> SciPy, Pandas, NumPy, MatplotLib,
* <b>Machine Learning intermediate experience:</b> ML for supervised modeling of time series data using neural networks. We use the Keras framework for TensorFlow in this notebook to create a Convolutional Neural Networks(CNN).
* <b>Space Weather introductory knowledge:</b> Basic familiarity of the Solar Wind and the Disturbance Storm Time activity index (<i>Dst</i>). For introductory materials on space weather and its effects on the technological systems we rely on, two resources are:
    * [NASA's Space Place](https://spaceplace.nasa.gov/spaceweather/),
    * [NOAA's Space Weather Prediction Center (SWPC)](https://www.swpc.noaa.gov/), in particular their community dashboards.

### Targeted Level
This notebook is targeted towards learners with beginner to intermediate experience in space weather topics, and intermediate experience in modeling time series data with neural networks.

### Learning Outcomes

By engaging in this notebook, the learner will:
1. Gain experience developing a regression model to predict the space weather disturbance-storm-time (<i>Dst</i>) index.
2. Build the functional benchmark CNN model from the NOAA MagNet competition.
2. Get familiar with using imperfect solar wind observations, as the modeling features.
3. Run the trained model on classical space weather events.

In [ ]:
<div class="alert alert-block alert-info">
<b>Info: </b>
In this notebook, you'll notice color-coded boxes, which provide hints, exercises, and warnings. Here is the color-coding breakdown:
</div>

* <span style="color:blue">Hint/Tip/Info: </span> Helpful context and guidance, as a blue alert-info box
* <span style="color:green">Exercise: </span> Interactive activity / exercise, as a green alert-success box
* <span style="color:orange">Be Aware: </span> Caution / Caveat, as a yellow alert-warn box
* <span style="color:red">Danger: </span> Conditions under which code may create an error, as a red alert-danger box

## Tutorial Material

## Background on Geospace Space Weather

Just like the terrestrial weather we are used to experiencing in our daily lives, weather also occurs in the space environment. If you'd like a general primer on space weather and its effects on the technological systems we rely on, check out [NASA's Space Place](https://spaceplace.nasa.gov/spaceweather/), as well as [NOAA's Space Weather Prediction Center (SWPC)](https://www.swpc.noaa.gov/), in particular their community dashboards.

![HELIO_GRAPHIC_URL](https://ngdc.noaa.gov/geomag/img/challenge-banner.png "HELIO")

## Background on the Geomagnetic Field

The efficient transfer of energy from solar wind into the Earth’s magnetic field causes geomagnetic storms. The resulting variations in the magnetic field increase errors in magnetic navigation. The disturbance-storm-time index, or <i>Dst</i>, is a measure of the severity of the geomagnetic storm.

As a key specification of the magnetospheric dynamics, the <i>Dst</i> index is used to drive geomagnetic disturbance models such as NOAA/NCEI’s High Definition Geomagnetic Model - Real-Time (HDGM-RT).
![HDGMRT_GRAPHIC_URL](https://www.ngdc.noaa.gov/geomag/HDGM/images/HDGM-RT_2003_storm_720p.gif "HDGM-RT")

In 2020-2021, NOAA and NASA conducted an international crowd sourced data science competition “MagNet: Model the Geomagnetic Field”:
https://www.drivendata.org/competitions/73/noaa-magnetic-forecasting/

Empirical models have been proposed as early as in 1975 to forecast <i>Dst</i> solely from solar-wind observations at the Lagrangian (L1) position by satellites such as NOAA’s Deep Space Climate Observatory (DSCOVR) or NASA's Advanced Composition Explorer (ACE). Over the past three decades, several models were proposed for solar wind forecasting of <i>Dst</i>, including empirical, physics-based, and machine learning approaches. While the ML models generally perform better than models based on the other approaches, there is still room to improve, especially when predicting extreme events. More importantly, we intentionally sought solutions that work on the raw, real-time data streams and are agnostic to sensor malfunctions and noise.


### Software
This notebook has been tested using the following environments:
* Google Colaboratory (Python 3.10.12) with no need to install additional packages.
    * CPU, GPU, TPU tested
* Anaconda (Python 3.9.16) with the following key package versions:
    * Keras TensorFlow 2.8.0
    * Pandas 1.5.3
    * Matplotlib 3.7.1
    * CPU, and GPU tested

## Modeling Task

The MagNet competition task was to develop models for forecasting <i>Dst</i> that push the boundary of predictive performance, under operationally viable constraints, using the real-time solar-wind (RTSW) data feeds from NOAA’s DSCOVR and NASA’s ACE satellites. Improved models can provide more advanced warning of geomagnetic storms and reduce errors in magnetic navigation systems. Specifically, given one week of data ending at t minus 1 minute, the model must forecast <i>Dst</i> at time t and t plus one hour.

<div class="alert alert-block alert-info">
<b>Question: </b> Can you describe the physical process between solar wind and ground geomagnetic disturbances? What is the <i>Dst</i> index primarily used for?
Roughly 85% of the time, near Earth is geomagnetically quiet. How might these infrequent solar wind events make modeling their predicted effects challenging? How might you make an accurate model with very few extreme events/samples?
</div>

## Data Notes

The target <i>Dst</i> values are measured by 4 ground-based observatories near the equator. These values are then averaged to provide a measurement of <i>Dst</i> for any given hour.
To ensure similar distributions between the training and test data, the data is separated into three non-contiguous periods. All data are provided with a `period` and `timedelta` multi-index which indicates the relative timestep for each observation within a period, but not the real timestamp. The period identifiers and timedeltas are common across datasets. Converting back from our index date and time to real geophysical date and time is as simple as adding the start date/time in the table below to the relative timestep provided with the data.


<div style="text-align: center">Table: Dataset Period Time Ranges</div>

| Period  | Beginning               | End                      |
|---------|-------------------------|--------------------------|
| train_a | 1998, 2, 16, '00:00:00' | 2001, 5, 31, '23:59:00'  |
| train_b | 2013, 6, 1, '00:00:00'  | 2019, 5, 31, '23:59:00'  |
| train_c | 2004, 5, 1, '00:00:00'  | 2010, 12, 31, '23:59:00' |
|  test_a | 2001, 6, 1, '00:00:00'  | 2004, 4, 30, '23:59:00'  |
|  test_b | 2011, 1, 1, '00:00:00'  | 2013, 5, 31, '23:59:00'  |
|  test_c | 2019, 6, 1, '00:00:00'  | 2020, 10, 31, '23:59:00' |


![Figure_Activity_and_Training_Splits.png](https://github.com/ai2es/tai4es-trustathon-2022/raw/space/space/notebook_figures/Figure_Activity_and_Training_Splits.png)
<i>Figure: Plot shows solar activity as the sunspot number (SSN) (red), the geomagnetic disturbance-storm-time (<i>Dst</i>) index (blue), and the data segments. The time range shown is January 1998 through December 2022, roughly corresponding to two solar-cycles. The data for periods “train_a”, “train_b”, and “train_c”  were provided to the participants as “public” data. The data for periods “test_a”, “test_b” and “test_c” were held back for “private” validation. This figure and the table preceding it have been adapted from Nair et al (manuscript in preparation).</i>

The competitors used the training part (“train_a”,”train_b” and “train_c”) data to develop and improve their models. When they submitted a model, the competition platform used the test data sets (“test_a”,”test_b” and “test_c”)  to calculate the accuracy of the model. The model evaluation was done separately for a public leaderboard and for a private leaderboard. The public leaderboard was openly accessible whereas the private leaderboard was restricted to the competition administrators. The  data from all of the training sets (a, b, and c) were used on the public leaderboard and private leaderboard. We randomly sampled rows to be included in the public and private leaderboard. Based on relative performance from the public leaderboard as a clue, the teams iterated their models. The final ranking of the models was done on the private leaderboard.

<b>Input data sources (i.e. features)</b>:

* Satellite measurements of the solar wind, including direction, speed, density and temperature, at 1-minute cadence.
* Position of the satellite used for solar wind measurements. The ACE and DSCOVR satellites are positioned just outside Earth's exosphere approximately 1% of the distance from Earth to Sun. As noted above, this is referred to as the Sun Earth L1 position.
* Number of sunspots on the Sun, measured monthly.

<div class="alert alert-block alert-info">
<b>Info:</b> Here is a short description of several of these inputs (features) observed by the ACE or DSCOVR satellites:

* bt - Interplanetary-magnetic-field magnitude (nT)
* bx_gsm - Interplanetary-magnetic-field X-component in geocentric solar magnetospheric (GSM) coordinate (nT)
* by_gsm - Interplanetary-magnetic-field Y-component in GSM coordinate (nT)
* bz_gsm - Interplanetary-magnetic-field Z-component in (GSM) coordinate (nT)
* density - Solar wind proton density (N/cm^3)
* speed - Solar wind bulk speed (km/s) flowing from Sun to Earth
* temperature - Solar wind ion temperature (Kelvin)
</div>

To get a feeling for the GSM coordinate reference frame:

The X-axis is oriented from the Earth to the Sun. The positive Z-axis is chosen to be in the same sense as the northern magnetic pole. And the Y-axis is defined to be perpendicular to the Earth's magnetic dipole so that the X-Z plane contains the dipole axis. For additional details, see [here](https://www.spenvis.oma.be/help/background/coortran/coortran.html#GSM).

To see how several of these parameters look during an example space weather event see [Figure 5](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2018SW001897#swe20716-fig-0005) of Redmon et al., 2018.


### Acquire Data

<div class="alert alert-block alert-info">
<b>Info: </b>
The competition discussed in this notebook used <i>public</i> data for development and the public leaderboard. A <i>private</i> dataset was kept internal during the competition for use in scoring by the organizers. Since the competition has passed, both datasets are publicly accessible from NOAA. We will build and evaluate the model using the competition's <i>public</i> data and evaluate storm event case studies using the competition's <i>private</i> data.
</div>

In [ ]:
%%capture
%%bash
#download the data
if [ ! -d "data" ]; then
  wget https://ngdc.noaa.gov/geomag/data/geomag/magnet/public.zip
  unzip public.zip
  wget https://ngdc.noaa.gov/geomag/data/geomag/magnet/private.zip
  unzip private.zip
  mkdir data
  mv public data
  mv private data
fi

#### Import  Input (Features) and Output (Labels) as Pandas DataFrames

In [ ]:

# load dataframes from csv files
data_folder = "data"
solar_cols = ["period", "timedelta", "bx_gsm", "by_gsm", "bz_gsm", "bt", "speed", "density", "temperature"]
solar_train = pd.read_csv(os.path.join(data_folder, "public", "solar_wind.csv"), usecols=solar_cols, dtype={"period": "category"})
dst_train = pd.read_csv(os.path.join(data_folder, "public", "dst_labels.csv"), dtype={"period": "category"})
sunspots_train = pd.read_csv(os.path.join(data_folder, "public", "sunspots.csv"), dtype={"period": "category"})
solar_test = pd.read_csv(os.path.join(data_folder, "private", "solar_wind.csv"), usecols=solar_cols, dtype={"period": "category"})
dst_test = pd.read_csv(os.path.join(data_folder, "private", "dst_labels.csv"), dtype={"period": "category"})
sunspots_test = pd.read_csv(os.path.join(data_folder, "private", "sunspots.csv"), dtype={"period": "category"})

### Data Exploration
<div class="alert alert-block alert-info">
<b>Info:</b> We'll explore our input (feature) and output (label) data to better understand it's data architecture, statistical description and basic input-output relationships.</div>

In [ ]:
# plot distributions
fig, axs = plt.subplots(4, 2, figsize=(20, 12))
for i, var in enumerate(["bx_gsm", "by_gsm", "bz_gsm", "bt", "speed", "density", "temperature", "smoothed_ssn"]):
  plt.sca(axs.flat[i])
  if var == "smoothed_ssn":
    sunspots_train["smoothed_ssn"].hist(bins=50)
  else:
    solar_train[var].hist(bins=50)
  axs.flat[i].set_title(var)

#### Feature Relationship

<div class="alert alert-block alert-success"> For merge the pandas Dataframe, users can assign the approach to merge:
left, right, outter, inner, cross. The approach is similar to the "join" function in SQL. In the following example, "left" means to preserve the key order and perform left outer join. </div>

<div class="alert alert-block alert-danger"> <b>Correlation matrix:</b>
Note that this is a slow command (several minutes) unless you have a GPU or TPU equivalent processor (then it's ~1 min).
Take advantage of Pandas DataFrame and merge our Input (Feature) and Output (Label) data.
i.e. merge, Solar Wind + Sunspots + Satellite Location + <i>Dst</i></div>

In [ ]:
solar_sunspots = solar_train.merge(sunspots_train, how='left')
corr = solar_sunspots.merge(dst_train, how='left').fillna(method="ffill").corr()

plt.figure(figsize=(10, 5))
plt.matshow(corr, cmap='seismic', vmin=-1, vmax=1, fignum=1)
plt.xticks(range(corr.shape[1]), corr.columns, rotation=90)
plt.gca().xaxis.tick_bottom()
plt.yticks(range(corr.shape[1]), corr.columns)


cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title("Feature Correlation Heatmap", fontsize=16)
plt.show()

del solar_sunspots
del corr

In [ ]:
#### Plot correlation with Dst

In [ ]:
# plot correlations with dst
fig, axs = plt.subplots(4, 2, figsize=(20, 20))
for i, var in enumerate(["bx_gsm", "by_gsm", "bz_gsm", "bt", "speed", "density", "temperature", "smoothed_ssn"]):
  if var == "smoothed_ssn":
    df = dst_train.merge(sunspots_train[[var,"period", "timedelta"]], 'left', on=["period", "timedelta"])
  else:
    df = dst_train.merge(solar_train[[var,"period", "timedelta"]], 'left', on=["period", "timedelta"])
  axs.flat[i].scatter(df[var].values, df['dst'], s=2)
  axs.flat[i].set_ylabel('dst')
  axs.flat[i].set_xlabel(var)
del df

### Feature Generation

The model uses input data in the
$(x, y, z)$ coordinate system, rather than the angular coordinate systems, so that the mean and standard deviation can be easily calculated. We use the GSM coordinate system, rather than GSE, since we found this gives better performance (see https://www.mssl.ucl.ac.uk/grid/iau/extra/local_copy/SP_coords/geo_sys.htm for details on these coordinate systems).

Periods where the temperature data is $< 1$ are excluded from the training data, since this is not physically realistic and suggests that sensors are malfunctioning. Missing data is filled
by linear interpolation (to reduce noise, the interpolation uses a smoothed rolling average,
rather than just the 2 points immediately
before and after the missing part).

Data is normalized by subtracting the median and dividing by the interquartile range (this approach is used rather
than the more usual mean and standard deviation because some variables have asymmetric distributions with
long tails).

To reduce the volume of data and the training time, data is aggregated  in 10-minute increments, taking the mean and standard deviation of each feature in
the increment. This could alternatively be done as the first layer of the neural network, but it's more efficient
to do it as a preprocessing step.

<div class="alert alert-block alert-success"> <b>Info: </b> Prepare data for training or prediction, by returning 10-minute aggregates. Aggregate solar_data into 10-minute intervals and calculate the mean and standard deviation. Merge with sunspot data. Normalize the training data and save the scaling parameters in a dataframe (these are needed to transform data for prediction). </div>

<div class="alert alert-block alert-success"> <b>Note: </b>This method modifies the input dataframes solar, sunspots, and dst. If you want to keep the original dataframes unaltered, pass copies, e.g. prepare_data_1_min(solar.copy(), sunspots.copy(), dst.copy()). </div>

In [ ]:
# function to preprocess data; we will use this to prepare the data for training
# and validation

def prepare_data_1_min(
    solar: pd.DataFrame,
    sunspots: Union[pd.DataFrame, float],
    dst: pd.DataFrame = None,
    norm_df=None,
    output_folder: str = None,
    coord_system: str = "gsm",
    output_freq: str = "10_minute",
) -> Tuple[pd.DataFrame, List[str]]:
    """
    If ``dst`` is ``None``, prepare dataframe of feature variables only for prediction
    using previously-calculated normalization scaling factors in ``norm_df``.
    If ``dst`` is not ``None``, prepare dataframe of feature variables and labels for
    model training. Calculate normalization scaling factors and
    save in ``output_folder``. In this case ``output_folder`` must not be ``None``.

    Args:
        solar: DataFrame containing solar wind data. This function uses the GSM
            co-ordinates.
        sunspots: DataFrame containing sunspots data, or float. If dataframe, will be
            merged with solar data using timestamp. If float, all rows of output data
            will use this number.
        dst: ``None``, or DataFrame containing the disturbance storm time (Dst) data,
        i.e. the labels for training
        norm_df: ``None``, or DataFrame containing the normalization scaling factors to
            apply
        output_folder: Path to the directory where normalization dataframe will be saved
        coord_system: either "gsm" or "gse"
        output_freq: "10_minute" or "hour", how to aggregate the output data


    Returns:
        solar: DataFrame containing processed data and labels
        train_cols: list of training columns
    """

    if not os.path.exists(output_folder):
      os.mkdir(output_folder)

    # convert timedelta
    solar["timedelta"] = pd.to_timedelta(solar["timedelta"])

    # merge data
    solar["days"] = solar["timedelta"].dt.days
    if isinstance(sunspots, pd.DataFrame):
        sunspots["timedelta"] = pd.to_timedelta(sunspots["timedelta"])
        sunspots.sort_values(["period", "timedelta"], inplace=True)
        sunspots["month"] = list(range(len(sunspots)))
        sunspots["month"] = sunspots["month"].astype(int)
        sunspots["days"] = sunspots["timedelta"].dt.days
        solar = pd.merge(
            solar,
            sunspots[["period", "days", "smoothed_ssn", "month"]],
            "left",
            ["period", "days"],
        )
    else:
        solar["smoothed_ssn"] = sunspots
    solar.drop(columns="days", inplace=True)
    if dst is not None:
        dst["timedelta"] = pd.to_timedelta(dst["timedelta"])
        solar = pd.merge(solar, dst, "left", ["period", "timedelta"])
    solar.sort_values(["period", "timedelta"], inplace=True)
    solar.reset_index(inplace=True, drop=True)

    # remove anomalous data (exclude from training and fill for prediction)
    solar["bad_data"] = False
    solar.loc[solar["temperature"] < 1, "bad_data"] = True
    solar.loc[solar["temperature"] < 1, ["temperature", "speed", "density"]] = np.nan
    for p in solar["period"].unique():
        curr_period = solar["period"] == p
        solar.loc[curr_period, "train_exclude"] = (
            solar.loc[curr_period, "bad_data"].rolling(60 * 24 * 7, center=False).max()
        )

    # fill missing data
    if "month" in solar.columns:
        solar["month"] = solar["month"].fillna(method="ffill")
    if coord_system == "gsm":
        train_cols = [
            "bt",
            "density",
            "speed",
            "bx_gsm",
            "by_gsm",
            "bz_gsm",
            "smoothed_ssn",
        ]
    elif coord_system == "gse":
        train_cols = [
            "bt",
            "density",
            "speed",
            "bx_gse",
            "by_gse",
            "bz_gse",
            "smoothed_ssn",
        ]
    else:
        raise ValueError(f"Invalid coord system {coord_system}")
    train_short = [c for c in train_cols if c != "smoothed_ssn"]
    for p in solar["period"].unique():
        curr_period = solar["period"] == p
        solar.loc[curr_period, "smoothed_ssn"] = (
            solar.loc[curr_period, "smoothed_ssn"]
            .fillna(method="ffill", axis=0)
            .fillna(method="bfill", axis=0)
        )
        # fill short gaps with interpolation
        roll = (
            solar[train_short]
            .rolling(window=20, min_periods=5)
            .mean()
            .interpolate("linear", axis=0, limit=60)
        )
        solar.loc[curr_period, train_short] = solar.loc[
            curr_period, train_short
        ].fillna(roll)
        solar.loc[curr_period, train_short] = solar.loc[
            curr_period, train_short
        ].fillna(solar.loc[curr_period, train_short].mean(), axis=0)

    # normalize data using median and inter-quartile range
    if norm_df is None:
        norm_df = solar[train_cols].median().to_frame("median")
        norm_df["lq"] = solar[train_cols].quantile(0.25)
        norm_df["uq"] = solar[train_cols].quantile(0.75)
        norm_df["iqr"] = norm_df["uq"] - norm_df["lq"]
    if output_folder is not None:
        norm_df.to_csv(os.path.join(output_folder, "norm_df.csv"))
    solar[train_cols] = (solar[train_cols] - norm_df["median"]) / norm_df["iqr"]

    if dst is not None:
        # interpolate target and shift target since we only have data up to t - 1 minute
        solar["target"] = (
            solar["dst"].shift(-1).interpolate(method="linear", limit_direction="both")
        )
        # shift target for training t + 1 hour model
        solar["target_shift"] = solar["target"].shift(-60)
        solar["target_shift"] = solar["target_shift"].fillna(method="ffill")
        assert solar[train_cols + ["target", "target_shift"]].isnull().sum().sum() == 0

    # aggregate features
    if output_freq == "10_minute":
        win = 10
    elif output_freq == "hour":
        win = 60
    else:
        raise ValueError("output_freq must be 10_minute or hour.")
    new_cols = [c + suffix for suffix in ["_mean", "_std"] for c in train_short]
    train_cols = new_cols + ["smoothed_ssn"]
    new_df = pd.DataFrame(index=solar.index, columns=new_cols)
    for p in solar["period"].unique():
        curr_period = solar["period"] == p
        new_df.loc[curr_period] = (
            solar.loc[curr_period, train_short]
            .rolling(window=win, min_periods=1, center=False)
            .agg(["mean", "std"])
            .values
        )
        new_df.loc[curr_period] = (
            new_df.loc[curr_period].fillna(method="ffill").fillna(method="bfill")
        )
    solar = pd.concat([solar, new_df], axis=1)
    solar[train_cols] = solar[train_cols].astype(float)

    # sample at output frequency
    solar = solar.loc[solar["timedelta"].dt.seconds % (win * 60) == 0].reset_index()

    return solar, train_cols

#### Data generator

We make a data generator to generate batches of input data and labels to feed to the keras model. Generating the data dynamically this way saves memory by avoiding the need to assemble all the data in a large array before training. (This large array would consume much more memory than the original data, because there is a lot of overlap in input data between subsequent prediction times, and thus a lot of repetition of the input data). Data is shuffled at the end of each training epoch, so each epoch's batches are different.

<div class="alert alert-block alert-success">The following data generator dynamically generates batches of training data for the keras model. Each batch consists of multiple time series sequences. See the keras documentation for more details:
https://keras.io/getting_started/faq/#what-do-sample-batch-and-epoch-mean
</div>


In [ ]:
# define data generator

class DataGen(tf.keras.utils.Sequence):

    def __init__(
        self,
        x: np.ndarray,
        valid_inds: np.ndarray,
        y: Optional[np.ndarray] = None,
        batch_size: int = 32,
        length: int = 24 * 6 * 7,
        shuffle: bool = True,
    ):
        """Construct the data generator.

        If ``y`` is not ``None``, will generate batches of pairs of x and y data,
        suitable for training. If ``y`` is ``None``, will generate batches of ``x``
        data only, suitable for prediction.

        ``x`` and ``y`` data must already be ordered by period and time. The training
        sample generated for an index ``i`` in ``valid_ind`` will have target ``y[i]``
        and ``x`` variables from rows ``(i - length + 1)`` to ``i`` (inclusive) of
        ``x``.  If there are multiple periods, there should be at least ``(length - 1)``
        data points before the first ``valid_ind`` in each period, otherwise the
        sequence for that valid_ind will include data from a previous period.

        Args:
            x: Array containing the x variables ordered by period and time
            y: ``None`` or array containing the targets corresponding to the ``x``
                variables
            batch_size: Size of training batches
            valid_inds: Array of ``int`` containing the indices which are valid
                end-points of training sequences (for example, we may set ``valid_inds``
                so it contains only the data points at the start of each hour).
            length: Number of data points in each sequence of the batch
            shuffle: Whether to shuffle ``valid_ind`` before training and after
                each epoch. For training, it is recommended to set this to ``True``, so
                each batch contains a varied sample of data from different times. For
                prediction, it should be set to ``False``, so that the predicted values
                are in the same order as the input data.
        """

        self.x = x
        self.y = y
        self.length = length
        self.batch_size = batch_size
        self.valid_inds = np.copy(valid_inds)
        self.shuffle = shuffle
        if self.shuffle:
            np.random.shuffle(self.valid_inds)

    def __get_y__(self):
        """Return the array of labels indexed by ``valid_ind``."""
        if self.y is None:
            raise RuntimeError("Generator has no y data.")
        else:
            return self.y[self.valid_inds]

    def __len__(self):
        """Return the number of batches in each epoch."""
        return int(np.ceil(len(self.valid_inds) / self.batch_size))

    def __getitem__(self, idx):
        """Generate a batch. ``idx`` is the index of the batch in the training epoch."""
        if (idx < self.__len__() - 1) or (len(self.valid_inds) % self.batch_size == 0):
            num_samples = self.batch_size
        else:
            num_samples = len(self.valid_inds) % self.batch_size
        x = np.empty((num_samples, self.length, self.x.shape[1]))
        end_indexes = self.valid_inds[
            idx * self.batch_size : (idx + 1) * self.batch_size
        ]
        for n, i in enumerate(end_indexes):
            x[n] = self.x[i - self.length : i, :]
        if self.y is None:
            return x
        else:
            y = self.y[end_indexes]
            return x, y

    def on_epoch_end(self):
        """Code to run at the end of each training epoch."""
        if self.shuffle:
            np.random.shuffle(self.valid_inds)



### Define and Build our CNN Model

#### Define CNN Model
The model consists of a set of convolutional layers which detect patterns at progressively longer time spans. At each convolutional
layer (except the last), a convolutional filter is applied having size 6 with stride 3, which reduces the size of the output data
relative to the input. (The last convolution has small input size, so it just convolves all its 9 inputs
together.) The earlier layers recognize low-level features on short time-spans, and these are outputs aggregated into higher-level
patterns spanning longer time ranges in the later layers. Cropping is applied at each layer which removes a few data points at the beginning at the
sequence to ensure the result will be exactly divisible by 6, so that the last application of the convolutional filter will
capture the data at the very end of the sequence. Following all the convolutional
layers is a layer which concatenates the last data point of each of the convolution outputs. This concatenation is then fed into a dense layer. The idea of taking the last data point of each convolution
is that it represents the patterns  at different timespans leading up to the prediction time: for example, the last data point
of the first layer gives the features of the hour before the prediction time, then the second layer gives
the last 6 hours, etc.

The architecture is somewhat similar to a widely used architecture for image segmentation, the U-Net introduced by Ronneberger, Fischer, and Brox (https://arxiv.org/abs/1505.04597). The U-Net consists of a "contracting path", a series of convolutional layers which condense the image, followed by an "expansive path" of up-convolution layers which expand the outputs back to the scale of the original image. Combining small-scale and large-scale features allows the network to make localized predictions that also take account of larger surrounding patterns.

The idea is also similar to the Temporal Convolutional Network described by Bai, Kolter, and Koltun (https://arxiv.org/abs/1803.01271); however their architecture uses residual (i.e. additive) connections to blend the low-level and high-level features, rather than concatenations.


In [ ]:
# define the structure of the neural network for 1-minute data

def define_model_cnn_1_min() -> Tuple[
    tf.keras.Model, List[np.ndarray], int, float, int
]:
    """
    Returns:
        model: keras model
        initial_weights: Array of initial weights used to reset the model to its
            original state
        epochs: Number of epochs
        lr: Learning rate
        bs: Batch size
    """

    inputs = tf.keras.layers.Input((6 * 24 * 7, 13))
    conv1 = tf.keras.layers.Conv1D(50, kernel_size=6, strides=3, activation="relu")(
        inputs
    )
    trim1 = tf.keras.layers.Cropping1D((5, 0))(
        conv1
    )  # crop from left so resulting shape is divisible by 6
    conv2 = tf.keras.layers.Conv1D(50, kernel_size=6, strides=3, activation="relu")(
        trim1
    )
    trim2 = tf.keras.layers.Cropping1D((1, 0))(conv2)
    conv3 = tf.keras.layers.Conv1D(30, kernel_size=6, strides=3, activation="relu")(
        trim2
    )
    trim3 = tf.keras.layers.Cropping1D((5, 0))(conv3)
    conv4 = tf.keras.layers.Conv1D(30, kernel_size=6, strides=3, activation="relu")(
        trim3
    )
    conv5 = tf.keras.layers.Conv1D(30, kernel_size=9, strides=9, activation="relu")(
        conv4
    )
    # extract last data point of previous convolutional layers (left-crop all but one)
    comb1 = tf.keras.layers.Concatenate(axis=2)(
        [
            conv5,
            tf.keras.layers.Cropping1D((334, 0))(conv1),
            tf.keras.layers.Cropping1D((108, 0))(conv2),
            tf.keras.layers.Cropping1D((34, 0))(conv3),
            tf.keras.layers.Cropping1D((8, 0))(conv4),
        ]
    )
    dense = tf.keras.layers.Dense(50, activation="relu")(comb1)
    output = tf.keras.layers.Flatten()(tf.keras.layers.Dense(1)(dense))
    model = tf.keras.Model(inputs, output)
    initial_weights = model.get_weights()
    epochs = 3
    lr = 0.00025
    bs = 32
    return model, initial_weights, epochs, lr, bs


#### Ensembling
The final model is an ensemble of 5 models with the same structure, trained on different subsets of the data (these are often called "cross-validation folds"). This is a common technique in machine learning. The idea is that each model only imperfectly captures the "true" relationship between the input and output variables, and partly fits to noise in the training data. But if we average several models, the random noise components will approximately cancel each other out, leaving a more accurate prediction of the true relationship.

The training data is segmented by month, and for each model 20% of months are excluded. The data is split by months rather than hours because successive hours are likely to be correlated, meaning test and train sets would be more similar, reducing the benefit of the ensemble. Separate models are trained for times  𝑡  and  𝑡+1 , yielding 10 models in total. To ensure each fold contains a mixture of extreme and moderate Dst periods, we order the months by average Dst, and then leave out every fifth month for each fold (e.g. in fold 0 we exclude months 4, 9, 14..., then in fold 1 we exclude months 1, 10, 15, ...).

For reproducibility, we set the numpy, keras, and python random seeds. Results may still not reproduce exactly due to hardware differences.

### Train Model
Here we execute the data preparation and training functions we defined earlier.

In [ ]:
# preprocess input data
output_folder = "trained_models_cnn"
solar_1_min, train_cols = prepare_data_1_min(
    solar_train, sunspots_train, dst_train, output_folder=output_folder, norm_df=None
)

In [ ]:
# train model
model_def_1_min = define_model_cnn_1_min
model, initial_weights, epochs, lr, bs = model_def_1_min()
train_on_prepared_data(
        solar_1_min,
        model,
        initial_weights,
        epochs,
        lr,
        bs,
        train_cols,
        5,
        output_folder,
        "minute",
        False
    )

### Evaluate Trained Model

#### define function to load saved models

In [ ]:


def load_models(
    input_folder: str, num_models: int,
) -> Tuple[List[tf.keras.Model], List[tf.keras.Model], pd.DataFrame]:
    """Define the model structure and load the saved weights of the trained models.

    Args:
        input_folder: Path to location where model weights are saved
        num_models: Number of models trained for each of ``t`` and ``t + 1`` (total
            number of models in folder should be ``2 * num_models``)

    Returns:
        model_t_arr: List of models for time ``t``
        model_t_plus_one_arr: List of models for time ``t + 1``
        norm_df: DataFrame of scaling factors to normalize the data
    """
    model_t_arr = []
    model_t_plus_one_arr = []
    for i in range(num_models):
        model = tf.keras.models.load_model(
            os.path.join(input_folder, "model_t_{}.h5".format(i))
        )
        model_t_arr.append(model)
        model = tf.keras.models.load_model(
            os.path.join(input_folder, "model_t_plus_one_{}.h5".format(i))
        )
        model_t_plus_one_arr.append(model)
    norm_df = pd.read_csv(os.path.join(input_folder, "norm_df.csv"), index_col=0)
    return model_t_arr, model_t_plus_one_arr, norm_df

In [ ]:
output_folder = "trained_models_cnn"
model_t_arr, model_t_plus_1_arr, norm_df = load_models(output_folder, 1)


#### Measure overall performance

We score the model on the private dataset, using the root mean squared error metric used in the competition. In the competition, the prediction function was called repeatedly on one week of input data at at time. Here we can speed up the prediction by using the data generator we defined above.

<div class="alert alert-block alert-success"> Make predictions for multiple times, which is faster than predicting one at a time.
Input data must be sorted by period and time and have a 1-minute frequency, and there must be at least 1 week of data before the first prediction time. </div>

#### Calculate the RMSE for ensembling model

In [ ]:
# define prediction function

def predict_batch(
    solar: pd.DataFrame,
    sunspots: pd.DataFrame,
    prediction_times: pd.DataFrame,
    model_t_arr: List[tf.keras.Model],
    model_t_plus_one_arr: List[tf.keras.Model],
    norm_df: pd.DataFrame,
    frequency: str,
    output_folder: str,
    comb_model: bool = False
) -> Tuple[pd.DataFrame, List, List]:
    """
    Args:
        solar: DataFrame containing solar wind data
        sunspots: DataFrame containing sunspots data
        prediction_times: DataFrame with a single column `timedelta` for which to make
            predictions. For each value ``t``, return predictions for ``t`` and
            ``t`` plus one hour.
        model_t_arr: List of models for time ``t``
        model_t_plus_one_arr: List of models for time ``(t + 1)``
        norm_df: Scaling factors to normalize the data
        frequency: frequency of the model, "minute", "hour" or "hybrid"
        output_folder: Path to the directory where normalisation dataframe will be saved
        comb_model: if ``True`` assumes a single model that predicts ``t`` and ``t+1``

    Returns:
        predictions: DataFrame with columns ``timedelta``, ``period``, ``prediction_t``
            and ``prediction_t_plus_1``
        t0_predictions_set: Dataframe containing the predicted t0 results from five individual models
        t1_predictions_set: Dataframe containing the predicted t1 results from five individual models

    """

    # validate input data
    solar["timedelta"] = pd.to_timedelta(solar["timedelta"])
    diff = solar["timedelta"].diff()
    diff.loc[solar["period"] != solar["period"].shift()] = np.nan
    valid_diff = solar["period"] == solar["period"].shift(1)
    if (frequency in ["minute", "hybrid"]) and np.any(
        diff.loc[valid_diff] != dt.timedelta(minutes=1)
    ):
        raise ValueError(
            "Input data must be sorted by period and time and have 1-minute frequency."
        )
    elif (frequency == "hour") and np.any(
        diff.loc[valid_diff] != dt.timedelta(hours=1)
    ):
        raise ValueError(
            "Input data must be sorted by period and time and have 1-hour frequency."
        )

    # add column to solar to indicate which times we must predict
    prediction_times["prediction_time"] = True
    solar = pd.merge(solar, prediction_times, on=["period", "timedelta"], how="left")
    solar["prediction_time"] = solar["prediction_time"].fillna(False)
    solar.sort_values(["period", "timedelta"], inplace=True)
    solar.reset_index(inplace=True, drop=True)

    # prepare data
    if frequency == "minute":
        solar, train_cols = prepare_data_1_min(
            solar.copy(), sunspots.copy(), output_folder=output_folder, norm_df=norm_df
        )
    elif frequency == "hour":
        solar, train_cols = prepare_data_hourly(
            solar.copy(), sunspots.copy(), norm_df=norm_df
        )
    elif frequency == "hybrid":
        solar, train_cols = prepare_data_hybrid(
            solar.copy(), None, sunspots.copy(), norm_df=norm_df, freq_for_1_min_data="10_minute"
        )
    else:
        raise ValueError(f"Invalid frequency {frequency}")

    # check there is 1 week of data before each valid time
    min_data_by_period = solar.groupby("period")["timedelta"].min().to_frame("min_time")
    min_data_by_period["min_prediction_time"] = (
        solar.loc[solar["prediction_time"]].groupby("period")["timedelta"].min()
    )
    min_data_by_period["data_before_first_prediction"] = (
        min_data_by_period["min_prediction_time"] - min_data_by_period["min_time"]
    )
    if min_data_by_period["data_before_first_prediction"].min() < dt.timedelta(days=7):
        raise RuntimeError(
            "There must be at least 1 week of data before the first prediction time in each period."
        )

    # valid_ind will be the endpoints of the sequences generated by the data generator;
    # these must be 1 minute/hour before the prediction time
    solar["valid_ind"] = solar["prediction_time"].fillna(False)

    # make prediction
    predictions = pd.DataFrame(prediction_times[["timedelta", "period"]].copy())
    valid_ind = solar.loc[solar["valid_ind"]].index.values
    sequence_length = 24 * 6 * 7 if frequency in ["minute", "hybrid"] else 24 * 7
    datagen = DataGen(
        solar[train_cols].values,
        valid_ind,
        y=None,
        batch_size=100,
        length=sequence_length,
        shuffle=False,
    )

    #Create the array for analyzing five models later.
    t0_predictions_set = []
    t1_predictions_set = []

    predictions["prediction_t"] = 0
    predictions["prediction_t_plus_1"] = 0
    if comb_model:
        for m in model_t_arr:
            predictions[["prediction_t", "prediction_t_plus_1"]] = np.array(m.predict(datagen))
    else:
        for m in model_t_arr:
            curr_model = np.array(m.predict(datagen)).flatten()
            t0_predictions_set.append(curr_model)
            predictions["prediction_t"] += curr_model
        predictions["prediction_t"] /= len(model_t_arr)

        for m in model_t_plus_one_arr:
            curr_model = np.array(m.predict(datagen)).flatten()
            t0_predictions_set.append(curr_model)
            predictions["prediction_t_plus_1"] += curr_model
        predictions["prediction_t_plus_1"] /= len(model_t_plus_one_arr)

    # restrict to allowed range
    predictions["prediction_t"] = np.maximum(
        -2000, np.minimum(500, predictions["prediction_t"])
    )
    predictions["prediction_t_plus_1"] = np.maximum(
        -2000, np.minimum(500, predictions["prediction_t_plus_1"])
    )

    return predictions, t0_predictions_set, t1_predictions_set


#### Calibrate time column

In [ ]:
dst_test["timedelta"] = pd.to_timedelta(dst_test["timedelta"])
# exclude times in the first week + 1 hour of dst_test
dst_test = dst_test.loc[dst_test["timedelta"] >= dt.timedelta(days=7, hours=1)].copy()


#### Measure RMSE

In [ ]:
# measure RMSE
predictions, t0_predictions_set, t1_predictions_set = predict_batch(
    solar_test.copy(), sunspots_test, dst_test, model_t_arr, model_t_plus_1_arr, norm_df, "minute", output_folder
)

#merge true data with prediction results
dst_test_1_min = pd.merge(dst_test, predictions, "left", ["timedelta", "period"])
dst_test_1_min["dst_t_plus_1"] = dst_test_1_min.groupby("period")["dst"].shift(-1)

aveModel_rmse_t = np.sqrt(
    mean_squared_error(dst_test_1_min["dst"].values, dst_test_1_min["prediction_t"].values)
)

valid_ind = dst_test_1_min["dst_t_plus_1"].notnull()
aveModel_rmse_t_plus_1 = np.sqrt(
    mean_squared_error(
        dst_test_1_min.loc[valid_ind, "dst_t_plus_1"].values,
        dst_test_1_min.loc[valid_ind, "prediction_t_plus_1"].values,
    )
)
print(f"RMSE for time t: {aveModel_rmse_t:0.2f}")
print(f"RMSE for time t+1: {aveModel_rmse_t_plus_1:0.2f}")

## Next steps

Congratulations on engaging with the learning objectives of this MagNet Chapter 1 CNN focused notebook--the benchmark from the NOAA MagNet competition. There is one additional <b>Chapter 2</b> notebook in the MagNet CNN series, on explainable AI (XAI) and includes using the model you've built to explore <i>Dst</i> prediction for user chosen storms.

There is an additional NCAI notebook in preparation for this MagNet series:
A LSTM notebook from Rob.Redmon.
As mentioned in an earlier section, this notebook's precursor is the [TAI4ES Space Weather LSTM Notebook](https://github.com/ai2es/tai4es-trustathon-2022/tree/main/space)

Additionally, a web search will provide other <i>Dst</i> modeling notebooks and publications using ML techniques.

## Examples in the community

For a comprehensive treatment of the need to build robust predictions of the <i>Dst</i> space weather storm indicator (e.g. for magnetic navigation applications), see Nair et al., 2023 and references therein:
* Nair et al., 2023 (<i>in press</i>) (TODO: Update with public URL as soon as available),

For a summary, see:
* https://www.drivendata.org/competitions/73/noaa-magnetic-forecasting/


## Data statement
The competition discussed above used <i>public</i> data for development and the public leaderboard. A <i>private</i> dataset was kept internal during the competition for use in scoring by the organizers. Since the competition has passed, both datasets are publicly accessible from NOAA.

All data used in this notebook are publicly available here:
* https://ngdc.noaa.gov/geomag/data/geomag/magnet/public.zip
* https://ngdc.noaa.gov/geomag/data/geomag/magnet/private.zip

## References

* Nair, M., Redmon, R.J., Young, L.Y., Chulliat, A., Trotta, B., Chung, C., Lipstein, G., Slavitt, I. (2023),"MagNet - a data-science competition to predict Disturbance Storm-time index (Dst) from solar wind data", Space Weather, <i>In Press</i>. (TODO: update with URL once available.)
* [CIRES GeoMag MagNet repository](https://github.com/liyo6397/MagNet/), TODO: update URL to new CIRES repo.
* [Trustworthy Artificial Intelligence for Environmental Science 2022 Summer School](https://www2.cisl.ucar.edu/events/tai4es-2022-summer-school), TAI4ES, accessed July 2022.
* [TAI4ES Space Weather Notebooks (LSTM, CNN)](https://github.com/ai2es/tai4es-trustathon-2022/tree/main/space), GitHub, accessed July 2022.
* [MagNet: Model the Geomagnetic Field](https://ngdc.noaa.gov/geomag/mag-net-challenge.html), NOAA, accessed March 2022.
* Chung, C. (2020), "HOW TO PREDICT DISTURBANCES IN THE GEOMAGENTIC FIELD WITH LSTMS - BENCHMARK", Blogpost, Accessed March 2022, Available Online: https://drivendata.co/blog/model-geomagnetic-field-benchmark/.
* DrivenData (2020), "MagNet: Model the Geomagnetic Field", Web Resource, Accessed March 2022, Available Online: https://www.drivendata.org/competitions/73/noaa-magnetic-forecasting/.
* [Interpretable Machine Learning by Christop Molnar](https://christophm.github.io/interpretable-ml-book/shap.html)
* Redmon, R. J., Seaton, D. B., Steenburgh, R., He, J., & Rodriguez, J. V. (2018). September 2017's geoeffective space weather and impacts to Caribbean radio communications during hurricane response. Space Weather, 16, 1190–1201. https://doi.org/10.1029/2018SW001897

## Metadata
 * Language / package(s):
     * Language: Python,
     * Packages: Keras Tensor Flow, Matplotlib, Numpy, Pandas, Scikit-learn
 * Scientific domain:
     * Space Weather, Geomagnetic modeling
 * Application keywords
     * Magnetic Navigation
 * Geophysical keywords
     * Disturbance Storm Index (<i>Dst</i>), Solar Wind
 * AI keywords
     * Convolutional Neural Networks (CNN)

## License

### Software and Content Description License
Software code created by U.S. Government employees is not subject to copyright in the United States (17 U.S.C. §105). The United States/Department of Commerce reserve all rights to seek and obtain copyright protection in countries other than the United States for Software authored in its entirety by the Department of Commerce. To this end, the Department of Commerce hereby grants to Recipient a royalty-free, nonexclusive license to use, copy, and create derivative works of the Software outside of the United States.

## Disclaimer

> This Jupyter notebook is a scientific product and is not official communication of the National Oceanic and Atmospheric Administration, or the United States Department of Commerce. All NOAA Jupyter notebooks are provided on an 'as is' basis and the user assumes responsibility for its use. Any claims against the Department of Commerce or Department of Commerce bureaus stemming from the use of this Jupyter notebook will be governed by all applicable Federal law. Any reference to specific commercial products, processes, or services by service mark, trademark, manufacturer, or otherwise does not constitute or imply their endorsement, recommendation or favoring by the Department of Commerce. The Department of Commerce seal and logo, or the seal and logo of a DOC bureau, shall not be used in any manner to imply endorsement of any commercial product or activity by DOC or the United States Government.